# Preparing the Dataset

In [5]:
pip install datasets==2.14.5


INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.8/163.8 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 10.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
  Attempting uninstall: dill
    Found existing installation: dill 0.3.8
    Uninstalling dill-0.3.8:
      Successfully uninstalled dill-0.3.8
  Attempting uninstall: multiprocess
    Found existing installation: multiprocess 0.70.16
    Uninstalling multiprocess-0.70.16:
      Successfully uninstalled multiprocess-0.70.16
  Attempting uninstall: datasets
    Found

In [1]:
from datasets import load_dataset

# Load the 2017 IWSLT dataset, English–French
dataset = load_dataset("iwslt2017", "iwslt2017-en-fr")

print(dataset)
print("Train size:", len(dataset["train"]))
print("Validation size:", len(dataset["validation"]))
print("Test size:", len(dataset["test"]))

# Access sentences
print(dataset["train"][0]["translation"]["en"])
print(dataset["train"][0]["translation"]["fr"])


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/232825 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/8597 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/890 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 232825
    })
    test: Dataset({
        features: ['translation'],
        num_rows: 8597
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 890
    })
})
Train size: 232825
Validation size: 890
Test size: 8597
Thank you so much, Chris. And it's truly a great honor to have the opportunity to come to this stage twice; I'm extremely grateful.
Merci beaucoup, Chris. C'est vraiment un honneur de pouvoir venir sur cette scène une deuxième fois. Je suis très reconnaissant.


In [6]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [7]:
# saving the dataset so I don't need to redownload it every time
dataset.save_to_disk("/content/drive/MyDrive/datasets/iwslt2017_en_fr")

Saving the dataset (0/1 shards):   0%|          | 0/232825 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/8597 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/890 [00:00<?, ? examples/s]

# Loading the Dataset

In [8]:
from datasets import load_from_disk

dataset = load_from_disk("/content/drive/MyDrive/datasets/iwslt2017_en_fr")
print(dataset)


DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 232825
    })
    test: Dataset({
        features: ['translation'],
        num_rows: 8597
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 890
    })
})


In [5]:
train_en = [sentence['translation']['en'] for sentence in dataset['train']]
train_fr = [sentence['translation']['fr'] for sentence in dataset['train']]

validation_en = [sentence['translation']['en'] for sentence in dataset['validation']]
validation_fr = [sentence['translation']['fr'] for sentence in dataset['validation']]

test_en = [sentence['translation']['en'] for sentence in dataset['test']]
test_fr = [sentence['translation']['fr'] for sentence in dataset['test']]

In [3]:
import numpy as np
first = np.random.randint(0, 200000)
second = np.random.randint(0, 200000)
third = np.random.randint(0, 200000)

print(train_en[first])
print(train_fr[first])

print("")

print(train_en[second])
print(train_fr[second])

print("")

print(train_en[third])
print(train_fr[third])

Sound, light: I'm going to draw the analogy.
Son, lumière: je vais vous montrer l'analogie.

Unfortunately the government schools don't function, so children are going to private schools today.
Malheureusement, les écoles publiques ne marchent pas, et donc les enfants étudient dans des écoles privées.

And that's essentially our main cost.
Et c'est notre coût principal.


# Implementing the Tokenizer